In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from numpy import array
from pickle import load,dump
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers import *
from keras.callbacks import ModelCheckpoint
import pydot
from numpy import argmax
from nltk.translate.bleu_score import corpus_bleu
import  pickle

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

dataset_path = r'C:\Users\Abhinav\Desktop\Masters Assignments\MLE project\UCM_captions'
split_path = 'drive/My Drive/MLE NEW/Dataset'

In [ ]:
from sklearn.model_selection import train_test_split
import os

def prepare_and_split_dataset(dataset_path, split_path):

    os.makedirs(os.path.join(split_path, 'train'), exist_ok=True)
    os.makedirs(os.path.join(split_path, 'validation'), exist_ok=True)
    os.makedirs(os.path.join(split_path, 'test'), exist_ok=True)

    # Get all file entries in the dataset directory and filter out directories
    all_entries = os.listdir(dataset_path)
    image_paths = [os.path.join(dataset_path, fname) for fname in all_entries if os.path.isfile(os.path.join(dataset_path, fname))]

    labels = [os.path.splitext(fname)[0] for fname in all_entries if os.path.isfile(os.path.join(dataset_path, fname))]

    # Split the dataset into training, validation, and testing sets
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        image_paths, labels, test_size=0.3, random_state=42
    )
    val_paths, test_paths, val_labels, test_labels = train_test_split(
        test_paths, test_labels, test_size=0.5, random_state=42
    )

    return train_paths, val_paths, test_paths, train_labels, val_labels, test_labels

In [ ]:
def move_files(file_paths, labels, target_dir):
    for path, label in zip(file_paths, labels):
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.copy(path, os.path.join(label_dir, os.path.basename(path)))

In [ ]:
train_paths, val_paths, test_paths, train_labels, val_labels, test_labels = prepare_and_split_dataset(dataset_path, split_path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Abhinav\\Desktop\\Masters Assignments\\MLE project\\UCM_captions'

In [ ]:
move_files(train_paths, train_labels, os.path.join(split_path, 'train'))
move_files(val_paths, val_labels, os.path.join(split_path, 'validation'))
move_files(test_paths, test_labels, os.path.join(split_path, 'test'))

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# import numpy as np

# # Create the encoder
# encoder = OneHotEncoder(sparse_output=False)

# all_labels = np.array(train_labels + val_labels + test_labels).reshape(-1, 1)
# encoder.fit(all_labels)

# # Transform the labels into one-hot encoded arrays
# train_labels_onehot = encoder.transform(np.array(train_labels).reshape(-1, 1))
# val_labels_onehot = encoder.transform(np.array(val_labels).reshape(-1, 1))
# test_labels_onehot = encoder.transform(np.array(test_labels).reshape(-1, 1))

In [ ]:
import os
import pickle
import numpy as np
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision.models import resnet152, ResNet152_Weights

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
def prepare_model(device):
    model = resnet152(weights=ResNet152_Weights.IMAGENET1K_V1)
    model.fc = torch.nn.Identity()
    model = model.to(device)
    return model

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
def extract_features(image_paths, model, preprocess, batch_size=32):
    dataset = ImageDataset(image_paths, transform=preprocess)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    features_list = []
    model.eval()
    with torch.no_grad():
        for images in dataloader:
            images = images.to(device)
            features = model(images)
            features_np = features.detach().cpu().numpy()
            # Change shape from (batch_size, 2048) to (batch_size, 1, 2048)
            features_np = features_np.reshape(features_np.shape[0], 1, features_np.shape[1])
            features_list.append(features_np)
    features = np.concatenate(features_list, axis=0)
    return features


In [ ]:
def save_features(features, labels, save_dir, set_name):
    os.makedirs(save_dir, exist_ok=True)
    filename = os.path.join(save_dir, f'{set_name}_features.pkl')
    with open(filename, 'wb') as f:
        pickle.dump({'features': features, 'labels': labels}, f)
    print(f'Features and labels saved to {filename}')

In [ ]:
model = prepare_model(device)

# Example usage, assuming train_paths, val_paths, test_paths, and their corresponding labels are defined
save_dir = '/home/a.kapavarapu/ondemand/Project/New_features_CNN'

train_features = extract_features(train_paths, model, preprocess)
save_features(train_features, train_labels, save_dir, 'train')

val_features = extract_features(val_paths, model, preprocess)
save_features(val_features, val_labels, save_dir, 'val')

test_features = extract_features(test_paths, model, preprocess)
save_features(test_features, test_labels, save_dir, 'test')

Features and labels saved to /home/a.kapavarapu/ondemand/Project/New_features_CNN/train_features.pkl
Features and labels saved to /home/a.kapavarapu/ondemand/Project/New_features_CNN/val_features.pkl
Features and labels saved to /home/a.kapavarapu/ondemand/Project/New_features_CNN/test_features.pkl


In [ ]:
import pickle

def transform_pkl_file(filename):
    # Load the original .pkl file
    with open(filename, 'rb') as file:
        data = pickle.load(file)

    # Transform the data
    new_data = {label: feature for label, feature in zip(data['labels'], data['features'])}

    # Save the transformed data
    with open(filename, 'wb') as file:
        pickle.dump(new_data, file)

train_filename = '/home/a.kapavarapu/ondemand/Project/New_features_CNN/train_features.pkl'
val_filename = '/home/a.kapavarapu/ondemand/Project/New_features_CNN/val_features.pkl'
test_filename = '/home/a.kapavarapu/ondemand/Project/New_features_CNN/test_features.pkl'

transform_pkl_file(train_filename)
transform_pkl_file(val_filename)
transform_pkl_file(test_filename)


In [ ]:
# filename = '/home/a.kapavarapu/ondemand/Project/New_features_CNN/train_features.pkl'
# with open(filename, 'rb') as file:
#     # Load the content of the file into a variable
#     data = pickle.load(file)
# data

In [ ]:
def read_captions(captions_file):
    captions_by_id = {}
    with open(captions_file, 'r') as file:
        for line_number, line in enumerate(file, start=1):
            line = line.strip()
            if not line:  # Skip empty lines
                continue
            parts = line.split(' ', 1)
            if len(parts) != 2:
                print(f"Warning: Line {line_number} formatted incorrectly: '{line}'")
                continue
            image_id, caption = parts
            if image_id not in captions_by_id:
                captions_by_id[image_id] = []
            captions_by_id[image_id].append(caption)
    return captions_by_id

captions_file = 'drive/My Drive/MLE_NEW/Captions/new_captions.txt'
captions_by_id = read_captions(captions_file)

FileNotFoundError: [Errno 2] No such file or directory: 'drive/My Drive/MLE_NEW/Captions/new_captions.txt'

In [ ]:
def get_image_ids_from_pkl(filename):
    with open(filename, 'rb') as file:
        data = pickle.load(file)
    return list(data.keys())

train_ids = get_image_ids_from_pkl('/home/a.kapavarapu/ondemand/Project/New_features_CNN/train_features.pkl')
val_ids = get_image_ids_from_pkl('/home/a.kapavarapu/ondemand/Project/New_features_CNN/val_features.pkl')
test_ids = get_image_ids_from_pkl('/home/a.kapavarapu/ondemand/Project/New_features_CNN/test_features.pkl')

In [ ]:
def filter_captions_for_set(image_ids, captions_by_id):
    captions = []
    for image_id in image_ids:
        if image_id in captions_by_id:
            for caption in captions_by_id[image_id]:
                captions.append(f"{image_id} {caption}")
    return captions

In [ ]:
def write_captions_to_file(captions, filename):
    with open(filename, 'w') as file:
        for caption in captions:
            file.write(f"{caption}\n")

write_captions_to_file(filter_captions_for_set(train_ids, captions_by_id), '/home/a.kapavarapu/ondemand/Project/Dataset/train_captions.txt')
write_captions_to_file(filter_captions_for_set(val_ids, captions_by_id), '/home/a.kapavarapu/ondemand/Project/Dataset/val_captions.txt')
write_captions_to_file(filter_captions_for_set(test_ids, captions_by_id), '/home/a.kapavarapu/ondemand/Project/Dataset/test_captions.txt')